## Load BOM

In [2]:
import os
import pandas as pd
import numpy as np
import requests
import time as t
import json
# from dotenv import load_dotenv
# load_dotenv()

BOM_path = f"../DAC_GainOutputStage7.csv"

BOM_df = pd.read_csv(BOM_path, delimiter = ";")
display(BOM_df)
print("Done")

,Qty,Value,Device,Package,Parts,Description,ARROW_PART_NUMBER,ARROW_PRICE-STOCK,DESCRIPTION,HEIGHT,MANUFACTURER,MANUFACTURER_NAME,MANUFACTURER_PART_NUMBER,MOUSER_PART_NUMBER,MOUSER_PRICE-STOCK,PROD_ID,Unnamed: 16
0,4,NaN,"BZX79-C16,113",DIOAD1165W56L425D185,"Z1, Z2, Z5, Z6","Nexperia Zener Diode, 16V +/-2 %, +/-5 % 500 m...",NaN,NaN,"Nexperia Zener Diode, 16V +/-2 %, +/-5 % 500 m...",mm,NaN,Nexperia,"BZX79-C16,113",NaN,NaN,NaN,NaN
1,2,NaN,CONN_06SILK_FEMALE_PTH,1X06,"J2, J13",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-08437,NaN
2,2,"0.1uF, 35V",MKS4B031002A00JC00,MKS4B031002A00JC00,"C2O, C2O1",Film Capacitors MKS 4 0.1 uF 50 VDC 2.5x7x10 P...,NaN,NaN,Film Capacitors MKS 4 0.1 uF 50 VDC 2.5x7x10 P...,7mm,NaN,WIMA,MKS4B031002A00JC00,505-MKS4B0312AJC,https://www.mouser.co.uk/ProductDetail/WIMA/MK...,NaN,NaN
3,8,100uF,UFW1H101MPD1TD,CAPPRD350W60D825H1300,"C11O, C12O, C13O, C14O, C15O, C24O, C25O, C26O",Aluminum Electrolytic Capacitors - Radial Lead...,UFW1H101MPD1TD,https://www.arrow.com/en/products/ufw1h101mpd1...,Aluminum Electrolytic Capacitors - Radial Lead...,13mm,NaN,Nichicon,UFW1H101MPD1TD,647-UFW1H101MPD1TD,https://www.mouser.co.uk/ProductDetail/Nichico...,NaN,NaN
4,10,10k,LR1F10K,0207/10,"R11O, R19O, R20O, R20O1, R20O2, R21O, R22O, R2...","Metal film resistor, 10K 0.6W TE Connectivity ...",NaN,NaN,"Metal film resistor, 10K 0.6W TE Connectivity ...",mm,NaN,TE Connectivity,LR1F10K,279-LR1F10K,https://www.mouser.co.uk/ProductDetail/TE-Conn...,NaN,NaN
5,4,15k,CPF215K000FKE14,0411/15,"R5, R11, R19, R20",VISHAY - CPF215K000FKE14 - Through Hole Resist...,NaN,NaN,VISHAY - CPF215K000FKE14 - Through Hole Resist...,mm,NaN,Vishay,CPF215K000FKE14,71-CPF2-F-15K-E3,https://www.mouser.com/ProductDetail/71-CPF2-F...,NaN,NaN
6,8,182,MF0207FTE52-182R,0207/10,"R12O, R14O, R16O, R18O, R23O, R24O, R25O, R26O",Metal Film Resistors - Through Hole 0.6W 1% 18...,NaN,NaN,Metal Film Resistors - Through Hole 0.6W 1% 18...,mm,Yageo,NaN,MF0207FTE52-182R,603-MF0207FTE52-182R,https://www.mouser.com/ProductDetail/YAGEO/MF0...,NaN,NaN
7,1,1935190,1935190,1935190,J1,"PCB terminal block, nominal current: 17.5 A, r...",NaN,NaN,"PCB terminal block, nominal current: 17.5 A, r...",11.55mm,NaN,Phoenix Contact,1935190,651-1935190,https://www.mouser.co.uk/ProductDetail/Phoenix...,NaN,NaN
8,2,1M*,LR1F1M0,0207/10,"R3O, R3O1",TE Connectivity Axial Metal Film Resistor 1m +...,NaN,NaN,TE Connectivity Axial Metal Film Resistor 1m +...,mm,NaN,TE Connectivity,LR1F1M0,279-LR1F1M0,https://www.mouser.co.uk/ProductDetail/TE-Conn...,NaN,NaN
9,6,1N4148-T50R,1N4148-T50R,DIOAD1068W53L380D172,"D1, D2, D3, D4, D5, D6","Diodes - General Purpose, Power, Switching Hi ...",NaN,NaN,"Diodes - General Purpose, Power, Switching Hi ...",mm,NaN,onsemi,1N4148-T50R,512-1N4148T50R,https://www.mouser.co.uk/ProductDetail/onsemi-...,NaN,NaN


Done


## Search Parts

In [3]:
def search_part(part_name):
    MOUSER_API_KEY = "c7b47461-c777-4dbc-8413-9bb69528b78f"
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    params = {
        'apiKey': MOUSER_API_KEY,
    }
    
    json_data = {
        'SearchByPartRequest': {
            'mouserPartNumber': part_name,
            'partSearchOptions': 'string',
            'mouserPaysCustomsAndDuties': False,
        },
    }
    
    response = requests.post('https://api.mouser.com/api/v1/search/partnumber', params=params, headers=headers, json=json_data)

    t.sleep(6)
    
    if response.status_code == 200:
        data = response.json()
    else:
        # Handle errors
        print(f'Error: {response.status_code} - {response.text}')
    
    df = pd.DataFrame()
    
    # Sort and append new data
    if data == {}:
        pass
    else:
        for part in data["SearchResults"]["Parts"]:
            df = pd.concat([df, pd.json_normalize(part)], ignore_index=True)

    columns_to_drop = ["ImagePath",
                     "AlternatePackagings",
                     "ProductDetailUrl",
                     "Reeling",
                     "MultiSimBlue",
                     "InfoMessages",
                     "SurchargeMessages",
                     "ProductCompliance",
                     "REACH-SVHC",
                     "RestrictionMessage",
                     "ROHSStatus",
                     "ProductAttributes",
                     "FactoryStock",
                     "AvailabilityOnOrder",
                    ]

    for column_to_drop in columns_to_drop:
        if column_to_drop in df.columns:
            df.drop([column_to_drop], axis = 1, inplace = True)
        else:
            pass

    try:
        df_exploded = df.explode("PriceBreaks").reset_index(drop=True)
        pricing_df = pd.json_normalize(df_exploded["PriceBreaks"])
        df_combined = pd.concat([df_exploded.drop(columns=["PriceBreaks"]), pricing_df], axis=1)
        df = df_combined.set_index(["MouserPartNumber", "Quantity"]).sort_index()
    except Exception as e:
        print(e)
        return
    
    for column in df.columns:
        try:
            df[column] = pd.to_numeric(df[column])
        except Exception as e:
            pass
    
    df = df[df["AvailabilityInStock"] > 0]
    df = df[df["Min"] == 1]
    df = df[df["Mult"] == 1]
    
    df = df.sort_values(by=['Quantity', 'Price'])
    # df =  df[df.index.isin([1.0], level=1)]
    return df.loc[[df.index.get_level_values(0)[0]]]

print("Done")

Done


In [4]:
results = pd.DataFrame()

for idx, part_name in enumerate(BOM_df["MOUSER_PART_NUMBER"].values):
    if part_name is not np.nan:
        results  = pd.concat([results, search_part(part_name)], axis = 0)
    else:
        results  = pd.concat([results, search_part(BOM_df.loc[idx, "Device"])], axis = 0)

display(results)

'PriceBreaks'
'PriceBreaks'
'PriceBreaks'


Availability  \
MouserPartNumber Quantity                   
771-BZX79-C16113 1         24368 In Stock   
                 10        24368 In Stock   
                 100       24368 In Stock   
                 500       24368 In Stock   
                 1000      24368 In Stock   
...                                   ...   
490-SJ-63033A    100          88 In Stock   
                 200          88 In Stock   
                 500          88 In Stock   
                 1000         88 In Stock   
                 2000         88 In Stock   

                                                                DataSheetUrl  \
MouserPartNumber Quantity                                                      
771-BZX79-C16113 1                                                       NaN   
                 10                                                      NaN   
                 100                                                     NaN   
                 500                                                     NaN   
                 1000                                                    NaN   
...                                                                      ...   
490-SJ-63033A    100       https://www.mouser.com/datasheet/3/6118/1/sj-6...   
                 200       https://www.mouser.com/datasheet/3/6118/1/sj-6...   
                 500       https://www.mouser.com/datasheet/3/6118/1/sj-6...   
                 1000      https://www.mouser.com/datasheet/3/6118/1/sj-6...   
                 2000      https://www.mouser.com/datasheet/3/6118/1/sj-6...   

                                                                 Description  \
MouserPartNumber Quantity                                                      
771-BZX79-C16113 1                Zener Diodes ZENER   SOD-27   .5W      16V   
                 10               Zener Diodes ZENER   SOD-27   .5W      16V   
                 100              Zener Diodes ZENER   SOD-27   .5W      16V   
                 500              Zener Diodes ZENER   SOD-27   .5W      16V   
                 1000             Zener Diodes ZENER   SOD-27   .5W      16V   
...                                                                      ...   
490-SJ-63033A    100       Phone Connectors audio jack, 6.35 mm, horizont...   
                 200       Phone Connectors audio jack, 6.35 mm, horizont...   
                 500       Phone Connectors audio jack, 6.35 mm, horizont...   
                 1000      Phone Connectors audio jack, 6.35 mm, horizont...   
                 2000      Phone Connectors audio jack, 6.35 mm, horizont...   

                                   Category LeadTime LifecycleStatus  \
MouserPartNumber Quantity                                              
771-BZX79-C16113 1             Zener Diodes  56 Days             NaN   
                 10            Zener Diodes  56 Days             NaN   
                 100           Zener Diodes  56 Days             NaN   
                 500           Zener Diodes  56 Days             NaN   
                 1000          Zener Diodes  56 Days             NaN   
...                                     ...      ...             ...   
490-SJ-63033A    100       Phone Connectors  91 Days             NaN   
                 200       Phone Connectors  91 Days             NaN   
                 500       Phone Connectors  91 Days             NaN   
                 1000      Phone Connectors  91 Days             NaN   
                 2000      Phone Connectors  91 Days             NaN   

                          Manufacturer ManufacturerPartNumber  Min  Mult  \
MouserPartNumber Quantity                                                  
771-BZX79-C16113 1            Nexperia          BZX79-C16,113    1     1   
                 10           Nexperia          BZX79-C16,113    1     1   
                 100          Nexperia          BZX79-C16,113    1     1   
                 500          Nexperia      